# Exercises on model ftting

This notebook is a set of exercises that you might like to try having worked through the model_fitting notebook which introduced you to finding an optimised 'fit' of a mathematical model to experimental data.

If you can cope with these you should be able to manaage any of the data fitting activities that you will come across in the rest of the module.

**SOLUTIONS are provided at the end, but it is highly recommended that you don't actually look at them until you've had a go yourself. To help in that respect, the exercises are setou at the top of the notebook and solutions given at the end. Do note these are really bate-bones solutions with little or no error checking.**


## Exercise 1 - Simple Hookes Law

The provided data file, 'hooke.csv' contains data collected from an experiment designed to find hooke's spring constant value for a test spring. The set up measured the total length of the spring ('l' in the file, measured in m) at a variety of forces ('f' in the file, measured as N).

Hookes law states that:

$$ F = -k x $$

where F is the applied force and x is the extension.

Use scipy.linregress() to work out 2 things:

1. The original, unloaded, length of the spring
2. Its spring constant, k

NOTE 
1. There is some uncertainty in the 'l' measurement - ignore that for the moment
2. The 'l' measurement is the TOTAL length of the spring at rest + its extension.

## Exercise 2 - Hooke's Law with errors

Repeat the above but get estimates for the accuracy of both values. Does the fact that we're told that the lengths are measured to +/- 5mm make any difference?


## Exercise 3 - 'g' on Mars

There is a 'standard' Physics methodolgy to measure the acceleration due to gravity (g) using a simple pendulum. This relates the time period (T) for a simple pendulum (mass on a string) to it's length, l (for small displacements). This is:

$$ {T} = {2\pi \sqrt{{l}\over{g}}} $$

We've been given exclusive access to a secret manned mission to mars and been provided with measured data that contains values for T (seconds) and l (metres) with associated errors, taken on the surface of Mars.

Rather than give you a data file, the measured data is:

`l = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
 T = [2.307051264, 2.825549203, 3.262663186, 3.647768336, 3.995930005, 4.316097701, 4.614102528]`

What is 'g' on Mars? 
NOTE: You could rearange the equation to get a straight line and use the code above. HOWEVER, that would be too easy (and uneccesary with appropriate coding)so do it by declaring the function as it appears above.



## Exercise 4 - 'g' with errors!

Due to operational difficulties, the length measurements taken were only accuarte to +/- 10cm and the timings to +/- 0.2 seconds.

Calculate the result, along with an uncertainty, using this information.


# Solutions

### Exercise 1

In [63]:
import pandas as pd
from scipy.stats import linregress

df = pd.read_csv('hooke.csv')
df.head()

,f,l
0,5,0.334
1,7,0.357
2,9,0.374
3,11,0.397
4,13,0.415


Now, we need to do a bit of rearranging of the formula to get the results we need. In our case, l = L + x (length of spring + extension), and we know that f = k\*x (we can ignore the -) so using this and rearranging we get:

$$ {l} = {\Big({{1} \over{k}}\Big)F + L} $$

So if we 'plot' l against F and do the regression fit, the gradient will be equal to 1/k and the intercept will be the unloaded spring length

In [64]:
m,c,r_val,p_val,sterr = linregress(df['f'],df['l'])

print(f'Intercept is {c}, Gradient is {m}')
print(f'So, we have: spring constant is {1/m:.4f} N/m and spring length is {c*100:.2f} cm')

Intercept is 0.27871428571428575, Gradient is 0.010857142857142857
So, we have: spring constant is 92.1053 N/m and spring length is 27.87 cm


### Exercise 2

In [65]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Define a simple, straight line model
def stLineFit(x, m, c):
    return (m*x + c)

df = pd.read_csv('hooke.csv')
df.head()

# Errors uncommenet to get error array
# err = [.005, .005, .005, .005, .005, .005, .005, .005]


# Do the fit
popt, pcov = curve_fit(stLineFit, df['f'],df['l'])
# Comment above and uncomment below to get fit including errors
# popt, pcov = curve_fit(stLineFit, df['f'],df['l'], sigma=err, absolute_sigma=True)

# Calculate the 'accuracy' of the returned parameters
perr = np.sqrt(np.diag(pcov)) # error values are on the covariance matrix diagonal

# Now we can print out the optimised fit parameters and 1-sigma estimates
print('fit parameter 1-sigma error')
print('***************************************************')
print (f'm = {popt[0]:.6f} +- {perr[0]:.7f}')
print (f'c = {popt[1]:.6f} +- {perr[1]:.7f}')
print('***************************************************')

print(f'Length of spring is {popt[1]*100:.1f} \u00B1 {perr[1]*100:.1f} cm')
print(f'Spring constant is {1/popt[0]:.0f} \u00B1 {perr[0]/popt[0]/popt[0]:.0f} N/m')

fit parameter 1-sigma error
***************************************************
m = 0.010857 +- 0.0002570
c = 0.278714 +- 0.0033011
***************************************************
Length of spring is 27.9 ± 0.3 cm
Spring constant is 92 ± 2 N/m


### Exercise 3

In [66]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Define a simple function relating l to T
def gFit(length, accel):
    return (2*np.pi*np.sqrt(length/accel))

# Data
l = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
T = [2.307051264, 2.825549203, 3.262663186, 3.647768336, 3.995930005, 4.316097701, 4.614102528]

# Do the fit
popt, pcov = curve_fit(gFit, l, T)

# Result!
print(f'g on Mars is {popt[0]:.2f} m/s')

g on Mars is 3.71 m/s


### Exercise 4

In [67]:
import numpy as np
from scipy import odr

# As before - but note the change in parameter order
def gFitODR(accel, length):
    return (2*np.pi*np.sqrt(length/accel))

# Data
l = [0.55, 0.8, 1.0, 1.27, 1.54, 1.65, 2.10]
T = [2.30, 2.9, 3.22, 3.65, 3.91, 4.36, 4.60]


# We also need the x and y errors 
l_err = [.1,.1,.1,.1,.1,.1,.1]
T_err = [.2,.2,.2,.2,.2,.2,.2] 

# Need to use scipy 'odr' model for fitting. 
model = odr.Model(gFitODR)
# Form an odr 'RealData' object
rdata = odr.RealData(l,T,sx=l_err,sy=T_err)

# We need an initial 'guess' 'g' - say 5
init_guess = [5] # Needs to be in list form

# Now set up the ODR, 
odr=odr.ODR(rdata, model, beta0=init_guess )
# Run it and get the results
result_outputs = odr.run()

# Extract the data we need
popt = result_outputs.beta
perr = np.sqrt(np.diag(result_outputs.cov_beta))

# print the results
print(f'g on Mars is {popt[0]:.1f} \u00B1 {perr[0]:.1f} m/s')

g on Mars is 3.8 ± 0.2 m/s
